In [40]:
import pandas as pd
import json
import requests 
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape

In [21]:
district_boundary_api = "https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/local-area-boundary/records?limit=20"
header =  {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

resp = requests.get(district_boundary_api, headers=header)
boundary_txt =  resp.text
boundary_data = json.loads(boundary_txt)

In [22]:
df = pd.read_csv('../data/processed_data/vancouver_real_estate.csv', index_col=0)
df.head()

,address,bedroom,bathroom,square_footage,price,postalCode,latitude,longitude,property_link,discrepancies
0,1001 Homer St #1901,1.0,1.0,505.0,658000.0,V6B 1M9,NaN,NaN,https://www.redfin.com/bc/vancouver/1001-Homer...,NaN
1,1003 Burnaby St #1106,1.0,1.0,411.0,485000.0,V6E 4R7,NaN,NaN,https://www.redfin.com/bc/vancouver/1003-Burna...,NaN
2,1005 W 7th Ave #303,1.0,1.0,981.0,899000.0,V6H 1B2,NaN,NaN,https://www.redfin.com/bc/vancouver/1005-W-7th...,NaN
3,1008 Cambie St #2906,1.0,NaN,802.0,998000.0,V6B 6J7,49.275743,-123.118,https://www.redfin.ca/bc/vancouver/1008-Cambie...,NaN
4,1009 Expo Blvd #1903,3.0,2.0,1146.0,1388000.0,V6Z 2V9,NaN,NaN,https://www.redfin.com/bc/vancouver/1009-Expo-...,NaN


In [36]:
def coords_to_polygon(coords):
    return Polygon(coords)

In [49]:
for i in range(boundary_data['total_count']):
    district_info = boundary_data['results'][i]
    district_name = district_info['name']
    print(district_name)

Dunbar-Southlands
Fairview
Mount Pleasant
Renfrew-Collingwood
West End
Arbutus Ridge
Grandview-Woodland
Kensington-Cedar Cottage
Killarney
Kitsilano
Strathcona
Sunset
West Point Grey
Downtown
Hastings-Sunrise
Kerrisdale
Marpole
Oakridge
Riley Park
South Cambie


IndexError: list index out of range

In [ ]:
boundary_data 

{'total_count': 22,
 'results': [{'name': 'Dunbar-Southlands',
   'geom': {'type': 'Feature',
    'geometry': {'coordinates': [[[-123.17016601562501, 49.24789047147971],
       [-123.17024993896484, 49.234703063032356],
       [-123.17870330810548, 49.234722136518705],
       [-123.17909240722656, 49.21680450346197],
       [-123.17908477783203, 49.21555709745609],
       [-123.17910003662108, 49.21555709745609],
       [-123.17975616455078, 49.21558761503423],
       [-123.18041229248047, 49.21563720609868],
       [-123.18106079101561, 49.215713500044004],
       [-123.18170166015624, 49.215812682172896],
       [-123.18232727050781, 49.2159347524854],
       [-123.18295288085936, 49.2160797109815],
       [-123.18355560302734, 49.216247557661184],
       [-123.18415069580078, 49.21643447782719],
       [-123.18472290039062, 49.216644286176795],
       [-123.1852798461914, 49.216873168012754],
       [-123.18581390380858, 49.21712112333501],
       [-123.1863327026367, 49.21739196684